In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json


In [2]:
TRAIN_DATA = [
    ('Facture FA2310-3910', {'entities': [(8, 19, 'Ref projet')]}),
    ('Date facturation : 18/10/2023', {'entities': [(19, 29, 'Date facture')]}),
    ('Date échéance : 19/10/2023', {'entities': [(16, 26, 'Date échéance')]}),
    ('Code client : CU2002-00627', {'entities': [(14, 26, 'code client')]}),
    ('Émetteur Adressé à', {'entities': []}),  # Aucune entité annotée
    ('JP & Co AJF TRANS', {'entities': [(1, 17, 'Nom ste/Enseigne/Raison sociale')]}),
    ('217, Boulevard Zerktouni, étage 3, numéro 11, Maarif Anassi 34 entrée 4 N° 41 Ahl Loughlam', {'entities': [(0, 90, 'Adresse')]}),
    ('20000 Casablanca Casablanca', {'entities': []}),  # Aucune entité annotée
    ('Tél.: +212522362219 - Fax: +212522362219 ICE: 001166918000075', {'entities': [(7, 19, 'Téléphone')]}),
    ('Email: info@jp.co.ma', {'entities': [(7, 20, 'email')]}),
    ('Web: http://www.jp.co.ma', {'entities': [(5, 24, 'site web')]}),
    ('Montants exprimés en Dirham', {'entities': []}),  # Aucune entité annotée
    ('Désignation TVA P.U. HT Qté Total HT', {'entities': []}),  # Aucune entité annotée
    ('ABOSI - Abonnement semestriel à la plateforme JPTRACK avec carte sim 0% 960,00 5 4 800,00', {'entities': [(8, 89, 'designation produit/service')]}),
    ('inclus', {'entities': [(0, 6, 'designation produit/service')]}),
    ('International', {'entities': [(0, 13, 'designation produit/service')]}),
    ('(d\'octobre 2023 à mars 2024 / 160 DH HT par mois par véhicule)', {'entities': [(0, 62, 'designation produit/service')]}),
    ('1.50328-A-13', {'entities': [(0, 12, 'designation produit/service')]}),
    # Facture 2
    ('Facture FA2310-3921', {'entities': [(0, 19, 'Ref projet')]}),
    ('Date facturation : 19/10/2023', {'entities': [(0, 29, 'Date facture')]}),
    ('Date échéance : 20/10/2023', {'entities': [(16, 26, 'Date échéance')]}),
    ('Code client : CU1903-0502', {'entities': [(14, 25, 'code client')]}),
    ('Émetteur Adressé à', {'entities': []}),  # Aucune entité annotée
    ('JP & Co TRANS BAYAMO', {'entities': [(0, 20, 'Nom ste/Enseigne/Raison sociale')]}),
    ('217, Boulevard Zerktouni, étage 3, numéro 11, Maarif Quartier al boustane, immeuble 4, appartement 2', {'entities': [(0, 100, 'Adresse')]}),
    ('20000 Casablanca 24000 El Jadida', {'entities': [(0, 32, 'Adresse')]}),
    ('Tél.: +212522362219 - Fax: +212522362219', {'entities': [(6, 20, 'Téléphone')]}),
    ('Email: info@jp.co.ma', {'entities': [(7, 20, 'email')]}),
    ('Web: http://www.jp.co.ma', {'entities': [(5, 24, 'site web')]}),
    ('Montants exprimés en Dirham', {'entities': []}),  # Aucune entité annotée
    ('Désignation TVA P.U. HT Qté Réduc. Total HT', {'entities': []}),  # Aucune entité annotée
    ('JPLIGHT - Boitier GPS JPLIGHT 20% 650,00 7 4 550,00', {'entities': [(0, 39, 'designation produit/service')]}),
    ('Règlement TTC par chèque', {'entities': []}),  # Aucune entité annotée
    ('Total TTC 4 550,00', {'entities': [(0, 13, 'Total TTC')]}),
    ('Conditions de règlement: A réception Total HT 4 550,00', {'entities': [(35, 46, 'Total HT')]}),
    ('Règlement TTC par chèque à l\'ordre de JP & Co envoyé à', {'entities': []}),  # Aucune entité annotée
    ('217, Boulevard Zerktouni, étage 3, numéro 11, Maarif', {'entities': [(0, 52, 'Adresse')]}),
    ('20000 Casablanca', {'entities': [(0, 16, 'Adresse')]}),
    ('Règlement par virement sur le compte bancaire suivant:', {'entities': []}),  # Aucune entité annotée
    ('Banque: BMCI', {'entities': [(8, 12, 'Banque')]}),
    ('Numéro de compte: 011780000047210000445396', {'entities': [(15, 45, 'Numéro de compte')]}),
    ('Adresse: AGENCE SOUKTANI - ANG RUE JEAN JAURES ET BD SOUKTANI 20000 CASABLNCA', {'entities': [(9, 77, 'Adresse')]}),
    ('MAROC', {'entities': [(0, 4, 'Adresse')]}),
    ('Nom du propriétaire du compte: JP&Co', {'entities': [(31, 36, 'Nom du propriétaire du compte')]}),
    ('Code IBAN: M640 1178 0000 0472 1000 0445 396', {'entities': [(9, 49, 'Code IBAN')]}),
    ('Code BIC/SWIFT: BMCEMAMC', {'entities': [(0, 20, 'Code BIC/SWIFT')]}),
    ('Société à Responsabilité Limitée (S.A.R.L) - Capital de 700 000 MAD - R.C.: ', {'entities': []}),  # Aucune entité annotée
    # Facture 3
    ('Facture FA2310-3922', {'entities': [(8, 19, 'Ref projet')]}),
    ('Date facturation : 20/10/2023', {'entities': [(19, 29, 'Date facture')]}),
    ('Date échéance : 21/10/2023', {'entities': [(16, 26, 'Date échéance')]}),
    ('Code client : CU2105-0742', {'entities': [(14, 26, 'code client')]}),
    ('Émetteur Adressé à', {'entities': []}),  # Aucune entité annotée
    ('JP & Co KANOU', {'entities': [(1, 14, 'Nom ste/Enseigne/Raison sociale')]}),
    ('Avenue de l\'Union Africaine, 102', {'entities': [(0, 33, 'Adresse')]}),
    ('20000 Casablanca', {'entities': [(0, 16, 'Adresse')]}),
    ('Tél.: +212522362219 - Fax: +212522362219', {'entities': [(6, 20, 'Téléphone')]}),
    ('Email: info@jp.co.ma', {'entities': [(7, 20, 'email')]}),
    ('Web: http://www.jp.co.ma', {'entities': [(5, 24, 'site web')]}),
    ('Montants exprimés en Dirham', {'entities': []}),  # Aucune entité annotée
    ('Désignation TVA P.U. HT Qté Réduc. Total HT', {'entities': []}),  # Aucune entité annotée
    ('ABOSI - Abonnement semestriel à la plateforme JPTRACK avec carte sim 0% 960,00 5 4 800,00', {'entities': [(0, 89, 'designation produit/service')]}),
    ('inclus', {'entities': [(0, 6, 'designation produit/service')]}),
    ('International', {'entities': [(0, 13, 'designation produit/service')]}),
    ('(d\'octobre 2023 à mars 2024 / 160 DH HT par mois par véhicule)', {'entities': [(0, 62, 'designation produit/service')]}),
    ('1.50328-A-13', {'entities': [(0, 12, 'designation produit/service')]}),
    ('Règlement TTC par chèque', {'entities': []}),  # Aucune entité annotée
    ('Total TTC 4 800,00', {'entities': [(0, 13, 'Total TTC')]}),
    ('Conditions de règlement: A réception Total HT 4 800,00', {'entities': [(35, 46, 'Total HT')]}),
    # Facture 4
    ('Facture FA2310-3923', {'entities': [(8, 19, 'Ref projet')]}),
    ('Date facturation : 21/10/2023', {'entities': [(19, 29, 'Date facture')]}),
    ('Date échéance : 22/10/2023', {'entities': [(16, 26, 'Date échéance')]}),
    ('Code client : CU2002-00627', {'entities': [(14, 26, 'code client')]}),
    ('Émetteur Adressé à', {'entities': []}),  # Aucune entité annotée
    ('JP & Co KHAL', {'entities': [(1, 11, 'Nom ste/Enseigne/Raison sociale')]}),
    ('Rue d\'Alger, 27', {'entities': [(0, 15, 'Adresse')]}),
    ('20000 Casablanca', {'entities': [(0, 16, 'Adresse')]}),
    ('Tél.: +212522362219 - Fax: +212522362219', {'entities': [(6, 20, 'Téléphone')]}),
    ('Email: info@jp.co.ma', {'entities': [(7, 20, 'email')]}),
    ('Web: http://www.jp.co.ma', {'entities': [(5, 24, 'site web')]}),
    ('Montants exprimés en Dirham', {'entities': []}),  # Aucune entité annotée
    ('Désignation TVA P.U. HT Qté Réduc. Total HT', {'entities': []}),  # Aucune entité annotée
    ('JPLIGHT - Boitier GPS JPLIGHT 20% 650,00 7 4 550,00', {'entities': [(0, 39, 'designation produit/service')]}),
    ('inclus', {'entities': [(0, 6, 'designation produit/service')]}),
    ('International', {'entities': [(0, 13, 'designation produit/service')]}),
    ('(d\'octobre 2023 à mars 2024 / 160 DH HT par mois par véhicule)', {'entities': [(0, 62, 'designation produit/service')]}),
    ('1.50328-A-13', {'entities': [(0, 12, 'designation produit/service')]}),
    ('Règlement TTC par chèque', {'entities': []}),  # Aucune entité annotée
    ('Total TTC 4 550,00', {'entities': [(0, 13, 'Total TTC')]}),
    ('Conditions de règlement: A réception Total HT 4 550,00', {'entities': [(35, 46, 'Total HT')]}),
]

In [3]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("fr") # load a new spacy model
#nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)


db.to_disk("./TRAIN_DATA.spacy") # save the docbin object

100%|██████████| 89/89 [00:00<00:00, 1589.30it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [5]:
!python -m spacy init fill-config base_config_fr.cfg config_fr.cfg

[+] Auto-filled config with all values
[+] Saved config
config_fr.cfg
You can now add your data and train your pipeline:
python -m spacy train config_fr.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
!python -m spacy train config_fr.cfg --output output --paths.train TRAIN_DATA.spacy --paths.dev TRAIN_DATA.spacy 

[+] Created output directory: output
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     62.53    0.00    0.00    0.00    0.00
 37     200        210.13   2778.03   95.16   96.72   93.65    0.95
 83     400         40.74    415.68   95.93   98.33   93.65    0.96
137     600         37.83    298.66   97.60   98.39   96.83    0.98
203     800         39.70    319.49   96.12   93.94   98.41    0.96
285    1000         42.75    385.47   97.64   96.88   98.41    0.98
385    1200         37.46    427.32   96.06   95.31   96.83    0.96
485    1400         30.63    440.58   96.06   95.31 

In [9]:
nlp2 = spacy.load(r"./output/model-best") #load the best model

In [12]:
with open("./FA2308-3865.txt", "r") as file:
    text_file = file.read()

# Process the text with spaCy NER
doc = nlp2(text_file)

# Print entities
for ent in doc.ents:
    print(ent.text, " ---->>" , ent.label_)

FA2308-3865  ---->> Ref projet
04/08/2023  ---->> Date facture
05/08/2023  ---->> Date échéance
CU1911-00591  ---->> code client
AdressÃ© Ã 
JP & Co BIBA TRAVAUX
217, Boulevard Zerktouni, Ã©tage 3, numÃ©ro 11, Maarif NÂ° 20 Rue  ---->> designation produit/service
+212522362219  ---->> Téléphone
info@jp.co.ma  ---->> email
http://www.jp.co.ma  ---->> site web
ABOTI - Abonnement trimestriel Ã  la plateforme JPTRACK avec carte sim 20%  ---->> designation produit/service
, septembre 2023, octobre 2023/ 40 DH HT par mois par
vÃ©hicule)  ---->> designation produit/service
ABO2I - Abonnement 2 mois Ã  la plateforme avec carte sim inclus 20%  ---->> designation produit/service
14463922  ---->> Téléphone
000009253000041  ---->> Téléphone
9792757  ---->> Téléphone
FFAA22330088--33886655
  ---->> Ref projet
CCUU11991111  ---->> Téléphone
Total HT  ---->> Total HT
Total TVA  ---->> Total HT
Total TTC 4  ---->> Total TTC
217, Boulevard Zerktouni, Ã©tage 3, numÃ©ro 11, Maarif  ---->> Adresse
AGENCE 

In [20]:
with open("./FA2308-3865.txt", "r") as file:
    text_file = file.read()

# Process the text with spaCy NER
doc = nlp2(text_file)

# List to store entities
entities_list = []

# Create dictionaries for entities
for ent in doc.ents:
    entity_dict = {
        "text": ent.text,
        "label": ent.label_
    }
    entities_list.append(entity_dict)


{'text': 'FA2308-3865', 'label': 'Ref projet'}
{'text': '04/08/2023', 'label': 'Date facture'}
{'text': '05/08/2023', 'label': 'Date échéance'}
{'text': 'CU1911-00591', 'label': 'code client'}
{'text': 'AdressÃ© Ã\xa0\nJP & Co BIBA TRAVAUX\n217, Boulevard Zerktouni, Ã©tage 3, numÃ©ro 11, Maarif NÂ° 20 Rue', 'label': 'designation produit/service'}
{'text': '+212522362219', 'label': 'Téléphone'}
{'text': 'info@jp.co.ma', 'label': 'email'}
{'text': 'http://www.jp.co.ma', 'label': 'site web'}
{'text': 'ABOTI - Abonnement trimestriel Ã\xa0 la plateforme JPTRACK avec carte sim 20%', 'label': 'designation produit/service'}
{'text': ', septembre 2023, octobre 2023/ 40 DH HT par mois par\nvÃ©hicule)', 'label': 'designation produit/service'}
{'text': 'ABO2I - Abonnement 2 mois Ã\xa0 la plateforme avec carte sim inclus 20%', 'label': 'designation produit/service'}
{'text': '14463922', 'label': 'Téléphone'}
{'text': '000009253000041', 'label': 'Téléphone'}
{'text': '9792757', 'label': 'Téléphone'}

In [21]:

# Save entities to a JSON file
output_file = "./entities_output.json"
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(entities_list, json_file, ensure_ascii=False, indent=4)

print("Entities extracted and saved to:", output_file)

Entities extracted and saved to: ./entities_output.json
